In [1]:
import os
from pathlib import Path

TXT_DATA = "..\\data\\adding"
XML_ROOT = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_zestaw2"


txt_path = Path(TXT_DATA)
txt_files = [f for f in txt_path.glob("*.txt")]
txt_files

[WindowsPath('../data/adding/21_09830A1_19_ROI_0_0_27_cls1_5.txt')]

In [2]:
from asap_loader.annotation.objects.rectangle import Rectangle
from asap_loader.annotation.objects.polygon import Polygon
from asap_loader.annotation.objects.point import Point
from asap_loader.annotation.objects.group import Group
import xml.etree.ElementTree as ET

from adding_adnotations.adding_to_xml import (
    get_info_from_filename,
    get_coord_txt_content,
    get_next_annotation_number,
    add_rectangle_annotation,
    add_annotation_groups
)

annotation_groups = {
    0: "komorki",
    1: "Mieloblasty",
    2: "Megakariocyty",
    3: "Atrefakty",
    4: "Naczynia",
    5: "Obszary",
}

class_map = {
    "komorki": 0,
    "Mieloblasty": 1,
    "Megakariocyty": 2,
    "Atrefakty": 3,
    "Naczynia": 4,
    "Obszary": 5,
}

all_colour = "#F4FA58"
annotation_colours = {
    "komorki": "#ffff00",
    "Megakariocyty": "#ff0000",
    "Mieloblasty": "#0055ff",
    "Artefakty": "#ff00ff",
    "Naczynia": "#0055ff",
    "Obszary": "#ffaa00",
}

for txt_file in txt_files:
    info = get_info_from_filename(txt_file.name)

    txt_contents = get_coord_txt_content(txt_file)

    xml_filename = f"{info['wsi']}.xml"
    xml_filepath = Path(XML_ROOT, xml_filename)

    tree = ET.parse(xml_filepath)
    root = tree.getroot()
    
    print( len(root.findall(".//Annotation")) )

    print(txt_contents)

    next_annotation_number = get_next_annotation_number(root)
    annotations = root.findall(".//Annotations")
    added_groups = set()

    for annotation_info in txt_contents:

        sth = add_rectangle_annotation(
            root.find(".//Annotations"),
            annotation_info,
            next_annotation_number,
            annotation_groups,
            all_colour,
        )
        added_groups.add(annotation_groups[annotation_info["cls_id"]])
        next_annotation_number += 1
        
    add_annotation_groups(
        root.find(".//AnnotationGroups"),
        added_groups,
        annotation_colours
        )

    print(len(root.findall(".//Annotation")))
    ET.indent(tree, space="  ")
    tree.write(f"{TXT_DATA}//example.xml", encoding="utf-8", xml_declaration=True)

{'wsi': '21_09830A1_19', 'roi_id': '0', 'cls_id': '1', 'other': ['0', '27', '5']}
3
[{'cls_id': 1, 'x_center': 57, 'y_center': 122, 'width': 117, 'height': 160}]
4


In [ ]:
""